In [2]:
import pandas as pd
from deepctr import SingleFeat
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from model import xDeepFM_MTL
import tensorflow as tf
from deepctr.input_embedding import preprocess_input_embedding


from deepctr.layers.core import MLP, PredictionLayer
from deepctr.layers.interaction import CIN
from deepctr.layers.utils import concat_fun
from deepctr.utils import check_feature_config_dict

import json
import gc
import numpy as np

In [3]:
try :
    del users_info
except:
    pass
with open("users.json",'r') as load_f:
    users_info = json.load(load_f)
for key in users_info.keys():
    users_info[key].pop('unfinish')
    users_info[key].pop('dislike')

In [4]:
try :
    del items_info
except:
    pass
with open("items.json",'r') as load_f:
    items_info = json.load(load_f)
for key in items_info.keys():
    items_info[key].pop('unfinish')
    items_info[key].pop('dislike')

In [5]:
data = pd.read_csv('./input/final_track2_train.txt', sep='\t', names=[
    'uid', 'user_city', 'item_id', 'author_id', 'item_city', 'channel', 'finish', 'like', 'music_id', 'did', 'creat_time', 'video_duration'])

In [6]:
del data['user_city']
del data['author_id']
del data['item_city']
del data['channel']
del data['music_id']
del data['did']
del data['creat_time']
del data['video_duration']

In [7]:
user_dim = 73973
item_dim = 4122688

In [8]:
user_vec = np.random.random((user_dim,8))
item_vec = np.random.random((item_dim,8))

In [9]:
uid_input = tf.keras.Input(shape=(1,))
item_id_input = tf.keras.Input(shape=(1,))

In [10]:
u_vec_emb = tf.keras.layers.Embedding(input_dim=(user_dim),output_dim=8,weights = [user_vec])
i_vec_emb = tf.keras.layers.Embedding(input_dim=(item_dim),output_dim=8,weights = [item_vec])

input_u_vec = tf.keras.layers.Flatten()(u_vec_emb(uid_input))
input_i_vec = tf.keras.layers.Flatten()(u_vec_emb(item_id_input))


In [11]:
# 历史上用户交互的item
his_like_i = tf.keras.Input(shape=(10,))
his_finish_i = tf.keras.Input(shape=(100,))
# his_dislike_i = tf.keras.Input(shape=(10,))
# his_unfinish_i = tf.keras.Input(shape=(10,))

# 历史上看过item的用户
his_like_u = tf.keras.Input(shape=(10,))
his_finish_u = tf.keras.Input(shape=(100,))
# his_dislike_u = tf.keras.Input(shape=(10,))
# his_unfinish_u = tf.keras.Input(shape=(10,))

#特征 embedding
his_like_i_v = i_vec_emb(his_like_i)
his_finish_i_v = i_vec_emb(his_finish_i)
his_like_u_v = u_vec_emb(his_like_u)
his_finish_u_v = u_vec_emb(his_finish_u)

In [12]:
his_like_i_feature = tf.keras.layers.LSTM(8, dropout=0.2, recurrent_dropout=0.2)(his_like_i_v)
his_finish_i_feature = tf.keras.layers.LSTM(8, dropout=0.2, recurrent_dropout=0.2)(his_finish_i_v)

In [13]:
his_like_u_feature = tf.keras.layers.LSTM(8, dropout=0.2, recurrent_dropout=0.2)(his_like_u_v)
his_finish_u_feature = tf.keras.layers.LSTM(8, dropout=0.2, recurrent_dropout=0.2)(his_finish_u_v)

In [14]:
final_feature = tf.keras.layers.concatenate([input_u_vec,input_i_vec,his_like_i_feature,
                                              his_finish_i_feature,his_like_u_feature,his_finish_u_feature],axis = -1)

In [15]:
fc1 = tf.keras.layers.Dense(20,activation='relu')(final_feature)
fc2 = tf.keras.layers.Dense(10,activation='relu')(fc1)
final_out  = tf.keras.layers.Dense(2,activation='sigmoid')(fc2)

In [16]:
inputs_list = [uid_input,item_id_input,his_like_i,his_finish_i,his_like_u,his_finish_u]
model = tf.keras.models.Model(inputs=inputs_list, outputs= [final_out])

In [17]:
data.head()

,uid,item_id,finish,like
0,57384,43192,0,0
1,3230,46822,1,0
2,1249,1209078,0,0
3,11928,1209079,0,0
4,51266,1209080,1,0


In [18]:
VALIDATION_FRAC = 0.2

In [157]:
sparse_features = ['uid',  'item_id']
data[sparse_features] = data[sparse_features].fillna('-1', )
target = ['finish', 'like']
mms = MinMaxScaler(feature_range=(0, 1))

sparse_feature_list = [SingleFeat(feat, data[feat].nunique())
                       for feat in sparse_features]

train_size = int(data.shape[0]*(1-VALIDATION_FRAC))
train = data.iloc[:train_size]
test = data.iloc[train_size:]

In [161]:
model.compile("adam", "binary_crossentropy",metrics=[auc])

In [168]:
model.fit_generator(data_generator(data,users_info,items_info, batch_size = 32),steps_per_epoch =15697872//32 )

Epoch 1/1
OK


ValueError: could not broadcast input array from shape (32,10) into shape (32)

In [160]:
# AUC for a binary classifier

def auc(y_true, y_pred):
#     print(y_true.shape)
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return tf.math.reduce_sum(s, axis=0)

#-----------------------------------------------------------------------------------------------------------------------------------------------------

# PFA, prob false alert for binary classifier

def binary_PFA(y_true, y_pred, threshold=tf.constant(0.5)):
    y_pred = tf.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = tf.math.reduce_sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = tf.math.reduce_sum(y_pred - y_pred * y_true)
    return FP/N

#-----------------------------------------------------------------------------------------------------------------------------------------------------

# P_TA prob true alerts for binary classifier

def binary_PTA(y_true, y_pred, threshold=tf.constant(0.5)):
    y_pred = tf.cast(y_pred >= threshold, 'float32')
    P = tf.math.reduce_sum(y_true)
    TP = tf.math.reduce_sum(y_pred * y_true)
    return TP/P

In [118]:
def get_item_info(iid,items_info):
    id_item = items_info.get(str(iid),{})
    if id_item == {}:
        return -1*np.ones(10),-1*np.ones(100)
    lake_length = 10-len(id_item['like'][:10])
#     print(lake_length)
#     print(id_item['like'][:10])
    if lake_length > 1:
        id_item['like'].extend([-1  for i in range(lake_length)])
#     print(id_item['like'][:10].extend([-1  for i in range(lake_length)]))
    like_info =  np.array(id_item['like'][:10])
    lake_length = 100-len(id_item['finish'][:100])
    if lake_length>1:
        id_item['finish'].extend([-1  for i in range(lake_length)])
    finish_info = np.array(id_item['finish'][:100])
    return like_info,finish_info

In [20]:
inputs_list

[<tf.Tensor 'input_1:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'input_2:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'input_3:0' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'input_4:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'input_5:0' shape=(?, 10) dtype=float32>,
 <tf.Tensor 'input_6:0' shape=(?, 100) dtype=float32>]

In [169]:
id_user = users_info.get(str(1),{})

In [170]:
id_user

{'finish': [1212837,
  815912,
  204823,
  233072,
  1209483,
  484515,
  1225800,
  96460,
  1322994,
  1209100,
  1210360,
  119834,
  1002773,
  66895,
  1425948,
  1251266,
  745173,
  82099,
  1343956,
  592182,
  1211573,
  1211519,
  428554,
  126899,
  79942,
  34237,
  2027215,
  3366503,
  3525299,
  3526430,
  325628,
  3574178,
  3527318,
  3526204,
  3315799,
  446497,
  4418,
  3483623,
  960201,
  3847358,
  25877,
  936800,
  3846068,
  3828363,
  3878852,
  682822,
  3039443,
  809730,
  16814,
  1586462,
  862887,
  343,
  3065616,
  11328,
  247375,
  3846522,
  1381311,
  3103775,
  1196371,
  1169296,
  1251032,
  3847126,
  709941,
  2460528,
  811736,
  1344568,
  1687435,
  858423,
  9086,
  3239348,
  680986,
  102018,
  53890,
  469085,
  1048248,
  3363060,
  59517,
  115604,
  414505,
  6161,
  375300,
  646550,
  286535,
  655666,
  1705819,
  768147,
  1047809,
  3271924,
  3448084,
  982633,
  3458439,
  786612,
  84895,
  3561,
  1,
  3237811,
  634091,


In [94]:
def get_user_info(uid,users_info):
    id_user = users_info.get(str(uid),{})
    if id_user == {}:
        return -1*np.ones(10),-1*np.ones(100)
    lake_length = 10-len(id_user['like'][:10])
#     print(lake_length)
#     print(id_user['like'][:10])
    if lake_length > 1:
        id_user['like'].extend([-1  for i in range(lake_length)])
#     print(id_user['like'][:10].extend([-1  for i in range(lake_length)]))
    like_info =  np.array(id_user['like'][:10])
    lake_length = 100-len(id_user['finish'][:100])
    if lake_length>1:
        id_user['finish'].extend([-1  for i in range(lake_length)])
    finish_info = np.array(id_user['finish'][:100])
    return like_info,finish_info

In [167]:
def data_generator(data,users_info,items_info, batch_size): 
    idx = np.arange(data.shape[0])
    np.random.shuffle(idx)
    batches = [idx[range(batch_size*i, min(len(data), batch_size*(i+1)))] for i in range(len(data)//batch_size+1)]
    uids = data.uid.values
    iids = data.item_id.values
    finishes = data.finish.values
    likes = data.like.values
    target = np.transpose(np.vstack((uids,iids)))
    print('OK')
    while True:
        for i in batches:
            xu = uids[i]
            xi = iids[i]
            yf = finishes[i]
            yl = likes[i]
            ulike = np.array([get_user_info(id_,users_info)[0] for id_ in xu])
            ufinish = np.array([get_user_info(id_,users_info)[1] for id_ in xu])
            ilike = np.array([get_item_info(id_,items_info)[0] for id_ in xi])
            ifinish = np.array([get_item_info(id_,items_info)[1] for id_ in xi])
            xx = np.array([xu,xi,ulike,ufinish,ilike,ifinish])
            yy = target[i]
            yield (xx, yy)